In [ ]:
# 모듈
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# 테스트 내역 주석처리
# NUM_WORDS = 10000
# # 학습데이터, 테스트데이터 설정
# (train_data, train_labels), (test_data, test_labels) = keras.datasets.imdb.load_data(num_words=NUM_WORDS)

In [ ]:
NUM_WORDS = 10000
# 학습데이터, 테스트데이터 설정(imbdb)
(train_data, train_labels), (test_data, test_labels) = keras.datasets.imdb.load_data(num_words=NUM_WORDS)

# 멀티-핫-벡터 함수 설정
def multi_hot_sequences(sequences, dimension):    
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0 
    return results
train_data = multi_hot_sequences(train_data, dimension=NUM_WORDS)
test_data = multi_hot_sequences(test_data, dimension=NUM_WORDS)

In [ ]:
# 학습데이터 시각화 확인
plt.plot(train_data[0])

In [ ]:
# 기준 모델 생성
baseline_model = keras.Sequential([
                                   keras.layers.Dense(16, activation='relu', input_shape=(NUM_WORDS,)),
                                   keras.layers.Dense(16, activation='relu'),
                                   keras.layers.Dense(1, activation='sigmoid')])
baseline_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'binary_crossentropy']) #손실함수 : binary_crossentropy(2개분류), sparse_categorical_crossentropy(3개분류)
baseline_model.summary()

In [ ]:
# 모델 학습(기준모델)
# RAM용량으로 인해 epochs -> 2 변경
baseline_history = baseline_model.fit(train_data, train_labels, epochs=2, batch_size=512, validation_data=(test_data, test_labels), verbose=2)

In [ ]:
# 작은 모델
smllaer_model = keras.Sequential([
                                   keras.layers.Dense(4, activation='relu', input_shape=(NUM_WORDS,)),
                                   keras.layers.Dense(4, activation='relu'),
                                   keras.layers.Dense(1, activation='sigmoid')])
smllaer_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'binary_crossentropy'])
smllaer_model.summary()

In [ ]:
# 모델 학습(작은 모델)
# RAM용량으로 인해 epochs -> 2, verbose -> 0 
smllaer_history = smllaer_model.fit(train_data, train_labels, epochs=2, batch_size=512, validation_data=(test_data, test_labels), verbose=0)

In [ ]:
# 큰 모델 
bigger_model = keras.Sequential([
                                   keras.layers.Dense(512, activation='relu', input_shape=(NUM_WORDS,)),
                                   keras.layers.Dense(512, activation='relu'),
                                   keras.layers.Dense(1, activation='sigmoid')])
bigger_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'binary_crossentropy'])
bigger_model.summary()

In [ ]:
# 모델 학습(큰 모델)
# RAM용량으로 인해 epochs -> 2, verbose -> 0 
bigger_history = bigger_model.fit(train_data, train_labels, epochs=2, batch_size=512, validation_data=(test_data, test_labels), verbose=0)

In [ ]:
# 훈련손실, 검증 손실 그래프 시각화
def plot_history(histories, key='binary_crossentropy'):
    plt.figure(figsize=(16,10))
    for name, history in histories:
        val = plt.plot(history.epoch, history.history['val_' +key], '--', label=name.title()+'Val')
        plt.plot(history.epoch, history.history[key], color=val[0].get_color(), label=name.title()+'Train')

    plt.xlabel('Epoch')
    plt.ylabel(key.replace('_','').title())
    # 그래프별 라벨 설정
    plt.legend()

    plt.xlim([0, max(history.epoch)])
     
plot_history([('baseline', baseline_history),
              ('smaller',smllaer_history),
              ('bigger', bigger_history)])


In [ ]:
# l2정규화
l2_model = keras.models.Sequential([
                                    keras.layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu', input_shape=(NUM_WORDS,)),
                                    keras.layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu', input_shape=(NUM_WORDS,)),
                                    keras.layers.Dense(1, activation='sigmoid')])
l2_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'binary_crossentropy'])
l2_model_history = l2_model.fit(train_data, train_labels, epochs=2, batch_size=512, validation_data=(test_data, test_labels), verbose=0)

In [ ]:
plot_history([('baseline', baseline_history),
              ('l2',l2_model_history)])

In [ ]:
# 드롭 아웃 : 벡터의 원소중 랜덤하게 0으로 변환, 보통 0.2 ~ 0.5 사이 사용
dpt_model = keras.models.Sequential([
                                    keras.layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu', input_shape=(NUM_WORDS,)),
                                    keras.layers.Dropout(0.5),
                                    keras.layers.Dense(16, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu', input_shape=(NUM_WORDS,)),
                                    keras.layers.Dropout(0.5),
                                    keras.layers.Dense(1, activation='sigmoid')])                                    
dpt_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'binary_crossentropy'])
dpt_model_history = dpt_model.fit(train_data, train_labels, epochs=2, batch_size=512, validation_data=(test_data, test_labels), verbose=0)

In [ ]:
plot_history([('baseline', baseline_history),
              ('dropout',dpt_model_history)])